# Yersinia phage Phi R1-37

Stages of analysis are presented as were performed with the Yersinia phage

In [1]:
#required imports
import os
import pathlib
import re

import numpy as np
import pandas as pd

import Bio.SeqIO as SeqIO
import Bio.SeqUtils.ProtParam as bp

import matplotlib.pyplot as plt
import seaborn as sns

import ungin_functions as ung

In [2]:
os.chdir("/d/user6/tl003/thesis")

In [3]:
#storing proteomes of uracil-DNA phages
genome = "phages/phir1-37.gb"
df = ung.parse_uracil_genome(genome=genome)
print(df)

       ID                                           Sequence  Seq Length
0    g001  MKKNVCVITERDVVLAKRRKGCKLQTDVDVALTEMNNIRFDFRVII...          90
1    g002  MKILKATNPRWFAQSNSTFSKFAIRGATEVLSDWMDNAFSAFCQLY...          83
2    g003  MYITRRKKRFNTTVYYNIYLVPGKMLTVRDKNIYSVYKIFEKELDF...          82
3    g004  MKTIKARGFNKNKILDLTPIKETRRSAMDTLMRRILFIEVDISNHM...          72
4    g005  MKIVRISRKKDRDIGRLIIGHIFATYKELKGTLLFRYDYIGYKCKV...          62
..    ...                                                ...         ...
362  g363  MKTIRTIKHLPFDRYNEQMTKTFLLDQPISKIFMIRKDAEDFDNEN...          84
363  g364  MYISRTNEREMNFKRKSNPKTVREIYTNLLHDDFSLPKYKMYFGFE...          97
364  g365  MKALSIIDDEEREGSIRDYDYKILEPINEILEDLEEEIMISGIGVT...          75
365  g366  MRRLKGKVVGIDNLLRWDRQWGGRMWLEGSAFNDYDVLFKDTVLEF...          74
366  g367  MKIKKFKMKYEYVYHDWLQQNAEEIIWGYQLIKIDPRHFSDIKVEI...          61

[367 rows x 3 columns]


In [4]:
#generating fasta file storing all sequences
ung.write_df_to_fasta(df=df, output_file="uploads/all_seqs.fasta")

In [11]:
#BLAST search performed with PDB blastp to identify any proteins with close homologs
#parsing BLAST output
df_blasthits = ung.parse_blast_output(df=df, input_file="uploads/blast_output.txt")
print(df_blasthits)

       ID  BLAST_evalues  BLAST_seqids
48   g049   1.000000e-29         37.56
130  g131   3.000000e-06         39.44
141  g142   2.000000e-10         39.56
193  g194   3.000000e-60         35.03
212  g213   5.000000e-25         41.36
260  g261  4.000000e-105         36.76
273  g274   5.000000e-58         36.54
297  g298   4.000000e-10         43.24
308  g309   9.000000e-09         43.48
310  g311   2.000000e-50         37.60
326  g327   1.000000e-10         43.06


## Structure prediction of protein sequences

In [ ]:
#monomer prediction using esm_fold
#note that only 363 out of 367 were able to be analysed
#those missing are g083, g099, g294, g295

In [7]:
#first filter = confidence of predicted structure

#analysing confidence data output
ung.analyse_confidence(input_dir="/d/user6/tl003/thesis/monomer_prediction", df=df, scale=False)
os.chdir("/d/user6/tl003/thesis")
print(df)

#predictions were divided into 3 categories manually
#category 1: mean pLDDT > 50%
#category 2: mean pLDDT > 35%, with stretches > 60%, and length >~ 100
#category 3: remaining structures filtered out by this step
#split = 152, 54, 157

       ID                                           Sequence  Seq Length  \
0    g001  MKKNVCVITERDVVLAKRRKGCKLQTDVDVALTEMNNIRFDFRVII...          90   
1    g002  MKILKATNPRWFAQSNSTFSKFAIRGATEVLSDWMDNAFSAFCQLY...          83   
2    g003  MYITRRKKRFNTTVYYNIYLVPGKMLTVRDKNIYSVYKIFEKELDF...          82   
3    g004  MKTIKARGFNKNKILDLTPIKETRRSAMDTLMRRILFIEVDISNHM...          72   
4    g005  MKIVRISRKKDRDIGRLIIGHIFATYKELKGTLLFRYDYIGYKCKV...          62   
..    ...                                                ...         ...   
362  g363  MKTIRTIKHLPFDRYNEQMTKTFLLDQPISKIFMIRKDAEDFDNEN...          84   
363  g364  MYISRTNEREMNFKRKSNPKTVREIYTNLLHDDFSLPKYKMYFGFE...          97   
364  g365  MKALSIIDDEEREGSIRDYDYKILEPINEILEDLEEEIMISGIGVT...          75   
365  g366  MRRLKGKVVGIDNLLRWDRQWGGRMWLEGSAFNDYDVLFKDTVLEF...          74   
366  g367  MKIKKFKMKYEYVYHDWLQQNAEEIIWGYQLIKIDPRHFSDIKVEI...          61   

    Mean pLLDT  
0         None  
1         None  
2         None  
3         None  
4 

In [2]:
#second filter = structural similarity to known UngIns
#performed with tm-align local install
#bash tmalign_structures.sh -t template -q predicted_structures -o tmalign_output_1.txt

tm = pd.read_csv('uploads/tmalign_output_1.txt')
tm = tm.sort_values('TMScore',ascending=False)
tm_filt=tm[tm['TMScore'] > 0.5].reset_index(drop=True)
print(tm.describe())
print(tm_filt.describe())
ung.plot_tm(df=tm, filename="TM")

       Aligned length        RMSD       SeqID     TMScore
count      824.000000  824.000000  824.000000  824.000000
mean        46.354369    3.703932    0.073885    0.352559
std         14.233964    0.901139    0.041982    0.080700
min         14.000000    0.460000    0.000000    0.119780
25%         37.000000    3.200000    0.045000    0.304513
50%         46.000000    3.710000    0.070000    0.350795
75%         55.000000    4.340000    0.098000    0.405210
max         88.000000    5.620000    0.243000    0.577660
       Aligned length       RMSD      SeqID    TMScore
count       32.000000  32.000000  32.000000  32.000000
mean        53.218750   3.194687   0.078406   0.523643
std          6.690071   0.364877   0.038815   0.020086
min         44.000000   2.510000   0.000000   0.502210
25%         50.000000   2.950000   0.052250   0.510840
50%         51.500000   3.140000   0.079000   0.517085
75%         56.000000   3.405000   0.098750   0.524905
max         79.000000   4.230000   0.1

In [3]:
#known UngIns compared to each other
#bash tmalign_structures.sh -t template -q template -o tmalign_output_2.txt
tm = pd.read_csv('uploads/tmalign_output_2.txt')
ung.plot_template_comp(df=tm,filename="template_comp",templates=["8AIM_A", "8AIN_B", "8AIL_CF", "6XQI_A"])